## Callable & runnable objects (Chap 14)

#### Anonymous Functions (Procs)
- lambdas *are* procs, but with slightly different internal structures.

In [1]:
# intro
pr = Proc.new {puts "howdy from inside a proc."}
pr.call

howdy from inside a proc.


In [2]:
# procs vs blocks - not every code block serves as the basis for a proc.
# not a proc:
[1,2,3].each {|x| puts x*10} 

# proc:
def myproc(&block)
    block.call
end
myproc {puts "i'm a block or proc or something."}

# proc can be in a code block:
p = Proc.new {|x| puts x.upcase}
%w{larry moe curly}.each(&p)

10
20
30
i'm a block or proc or something.
LARRY
MOE
CURLY


["larry", "moe", "curly"]

- capturing a code block as a proc: the "phantom" proc creation step explains why the &-based syntax is needed. (Without the flag, Ruby has no way of knowing that you want to stop binding params to regular arguments, and instead performa a block-to-proc conversion.)
![capture](px/Selection_175.png)

In [3]:
def capture_block(&block)
    puts "got a block as a proc."
    block.call
end
capture_block {puts "inside a block."}

got a block as a proc.
inside a block.


In [4]:
# using procs for blocks
p = Proc.new { puts "howdy. i'm a proc argument, serving as a code block."}
capture_block(&p)

# FYI: the & operator here is a wrapper for to_proc. 

got a block as a proc.
howdy. i'm a proc argument, serving as a code block.


In [5]:
# Hash#to_proc
# typically used to run multiple lookups at once.

albums = { 
        1988 => "Straight Outta Compton", 
        1993 => "Midnight Marauders",
        1996 => "The Score", 2004 => "Madvillainy",
        2015 => "To Pimp a Butterfly" }

puts albums[2015] # single lookup
puts [1988,1996].map(&albums) # multi lookups
puts (1990..1999).map(&albums) # using a range
puts (1990..1999).map(&albums).compact

To Pimp a Butterfly
["Straight Outta Compton", "The Score"]
[nil, nil, nil, "Midnight Marauders", nil, nil, "The Score", nil, nil, nil]
["Midnight Marauders", "The Score"]


In [6]:
# generalizing to_proc
# gives an idea of the power of to_proc
class Person
    attr_accessor :name
    def self.to_proc
        Proc.new {|p| p.name}
    end
end
d = Person.new; d.name = "david"
j = Person.new; j.name = "joe"
puts [d,j].map(&Person)

["david", "joe"]


In [7]:
# Symbol#to_proc (here, :capitalize is a symbol)
%w{ moe larry curly }.map(&:capitalize)

["Moe", "Larry", "Curly"]

In [8]:
# procs as closures
def multiply_by(m)
    Proc.new {|x| puts x*m}
end
mult = multiply_by(10)
mult.call(12)

120


In [9]:
# notice how the Proc object carries it context with it. (a closure.)
def callme(pr)
    a = "irrelevant 'a' in method scope"
    puts a 
    pr.call
end
a = "'a' to be used in proc block"
pr = Proc.new {puts a}
pr.call
callme(pr)

'a' to be used in proc block
irrelevant 'a' in method scope
'a' to be used in proc block


In [10]:
# classic closure example: a counter
def make_counter
    n=0
    return Proc.new {n+=1}
end
c = make_counter; puts c.call; puts c.call
d = make_counter; puts d.call; puts c.call

1
2
1
3


In [11]:
# proc params & arguments
p1 = Proc.new {|x| puts "called with argument #{x}"}
p2 = Proc.new {|x| p x}
p1.call(100)
p2.call

called with argument 100
nil


#### creating functions with lambda and ->
- lambdas expect correct number of arguments
- lambdas require explicit creation
- lambdas with embedded "return" trigger immediate exit.

In [12]:
# intro
lam = lambda {puts "i'm a lambda."}
lam.call

def rtn_test
    l = lambda {return}
    l.call
    puts "still here"
    p = Proc.new {return}
    p.call
    puts "you should'nt see this"
end
rtn_test

i'm a lambda.
still here


In [13]:
# "stabby lambda" (->)
lam =  ->      {puts "howdy."}
mult = ->(x,y) {x*y}

lam.call; mult.call(3,4)

howdy.


12

#### Methods as objects
- Methods don’t present themselves as objects until you tell them to. Treating methods as objects involves objectifying them.

In [14]:
# capturing method objects
class C
    def talk
        puts "method grabber: self is #{self}."
    end
end
c = C.new; meth = c.method(:talk); puts meth.owner
meth.call

C
method grabber: self is #<C:0x00005587913ef3a0>.


In [15]:
# unbinding method from an object, rebinding it to another
class D<C
end
d = D.new; unbound = meth.unbind; unbound.bind(d).call

method grabber: self is #<D:0x0000558791366050>.


In [16]:
# why methods as objects
# suppose i've got a class hierarchy where a method is redefined?

# TODO: SOLVE SUPERCLASS MISMATCH ERROR

#### Eval methods

In [17]:
# eval: arbitrary strings as code
eval("2+2")

4

In [18]:
# eval dangers
# - VERY EASY to allow destructive behavior, ie "rm -rf /*"
# Object.tainted? flags potentially insecure data, ie from "gets".
x = "some bogus user input" # instead of using gets
puts x.tainted?

# $SAFE: 0 by default; 
# 1 = raises SecurityError is eval tried on tainted data
puts $SAFE

false


(pry):125: warning: $SAFE will become a normal global variable in Ruby 3.0


0


In [19]:
# instance_eval - evals string or code block 
# changes self to be the call receiver
p self
a = []; a.instance_eval {p self}

# usually used to peek into another object's private data,
# esp instance variables. (considered impolite.)
class C
    def initialize
        @x=1
    end
end
c = C.new; c.instance_eval {puts @x}

# another use case: simplified assignments
class Person
    def initialize(&block)
        instance_eval(&block)
    end
    def name(name=nil)
        @name ||= name
    end
    def age(age=nil)
        @age ||= age
    end
end
joe = Person.new do 
    name "joe"
    age 37
end

main
[]
1


#<Person:0x0000558790c1d7a8 @name="joe", @age=37>

In [20]:
# class_eval (aka module_eval)
c = Class.new
c.class_eval do
    def dothis
        puts "created in class eval"
    end
end
c_instance = c.new; c_instance.dothis

created in class eval


#### Threads
- Ruby’s threads allow you to do multiple tasks via time sharing (one thread executes instructions, passes control to the next thread, etc.)
- Ruby will try to use native OS threading & will otherwise fall back to "green threads" (implemented inside the interpreter).

- Parallelism and concurrency are NOT identical. Parallelism ensures that two or more tasks can run at the **exact** same time; concurrency only ensures a sort of time sharing that allows for faster processing. 

- Ruby uses a Global Interpreter Lock (GIL). It helps to ensure thread safety in concurrent programming but prevents parallelism.

- In this section, we’ll discuss concurrency as it refers to the execution of multiple threads, regardless of whether they run in parallel. Actual behavior will depend on your hardware and Ruby implementation.

In [21]:
# intro
Thread.new do 
    puts "starting..."; sleep 1
    puts "...ending."
end
puts "outside thread."

starting...
outside thread.


In [22]:
# join: allowing thread to finish 
t = Thread.new do 
    puts "starting..."; sleep 1
    puts "...ending."
end
puts "outside thread"; t.join

outside thread
starting...
...ending.
...ending.


#<Thread:0x00005587919e7780 (pry):171 dead>

In [23]:
# killing, stopping, starting threads
# example use case: exception occurs inside a thread.
puts "trying to read file"
t = Thread.new do 
    (0..2).each do |n|
        begin
            File.open("part0#{n}") do |f|
                text << f.readlines
            end
            rescue Errno::ENOENT
                puts "msg from thread: failed on n=#{n}"
            Thread.exit
        end
    end
end
t.join; puts "done"

trying to read file
msg from thread: failed on n=0
done


In [24]:
# fibers == re-entrant code blocks == basis of enumerators.
f = Fiber.new do 
    puts "howdy"
    Fiber.yield
    puts "nice day"
    Fiber.yield
    puts "bye"
end
f.resume
puts "back to the fiber"
f.resume
puts "one last msg from the fiber"
f.resume
puts "that's it."

howdy
back to the fiber
nice day
one last msg from the fiber
bye
that's it.


In [25]:
# threaded date server
# DO OUTSIDE JUPYTER

In [26]:
# chat server with sockets & threads
# DO OUTSIDE JUPYTER

In [27]:
# threads & variables
a=1
Thread.new {a=2}
a

1

In [28]:
t = Thread.new {Thread.stop; a=3}
puts a
t.run
puts a

2
2


In [29]:
# thread keys - storage hash for thread-specific values
t = Thread.new do 
    Thread.current[:message]="howdy"
end
t.join
p t.keys
puts t[:message]

# note: even though thread is "dead", you can still query the keys.
# or you can use fetch to get values & assign defaults.

t = Thread.new do 
    Thread.current[:message]="marco."
end
t.join
p t.fetch(:message, "polo.")
p t.fetch(:msg,     "polo.")
# p t.fetch(:msg) <== this raises a KeyError

[:message]
howdy
"marco."
"polo."


"polo."

In [30]:
# rock-paper-scissors
# TODO

#### System commands

In [31]:
# system method, exec method, backticks
puts system("date")
puts $? # process ID just completed, and status.
puts system("bogus")
puts $?

true
pid 24417 exit 0

pid 24418 exit 127


In [32]:
# backticks -- returns program output
puts `date`
puts `ls`

Tue Sep 22 12:57:07 CDT 2020

ch01-bootstrap.ipynb
ch02-objs-meths-locals.ipynb
ch03-classes.ipynb
ch04-modules.ipynb
ch05-self.ipynb
ch07-essentials.ipynb
ch08-scalars.ipynb
ch09-collections-containers.ipynb
ch10-enumerables.ipynb
ch11-regexes.ipynb
ch12-file-io.ipynb
ch13-object-individualization.ipynb
ch14-callables-runnables.ipynb
ch15-callbacks-hooks-introspection.ipynb
ch16-functional-programming.ipynb
px
well-grounded-rubyist.ipynb



In [33]:
# %x = alias for backticks
%x("ls")

"ch01-bootstrap.ipynb\nch02-objs-meths-locals.ipynb\nch03-classes.ipynb\nch04-modules.ipynb\nch05-self.ipynb\nch07-essentials.ipynb\nch08-scalars.ipynb\nch09-collections-containers.ipynb\nch10-enumerables.ipynb\nch11-regexes.ipynb\nch12-file-io.ipynb\nch13-object-individualization.ipynb\nch14-callables-runnables.ipynb\nch15-callbacks-hooks-introspection.ipynb\nch16-functional-programming.ipynb\npx\nwell-grounded-rubyist.ipynb\n"

In [34]:
# exec - same behavior as system and backticks
# replaces current process with new shell.
# (system & backticks uses "fork" to run command in a subshell.)

In [35]:
# communicating with programs - open, popen3
# TODO